In [25]:
%config Completer.use_jedi = False



> To live! like a tree alone and free,
> and like a forest in solidarity...
> Nazim Hikmet

<center> <h1> Random Forests (Yes! No Forest Image) </h1> </center>



__Objectives__

- Introduction of 'bagging' procedure.

- Identifying the need for bootstrapping for decision trees.

- Comparing Random forests and bagging methods

- Evaluating a model by random forest model

# Review: Bootstrapping


<img src= "img/bootstrap1.png" style="height:250px">


# Bagging


Let's us one more time recall that if $Z_{1}, \cdots, Z_{n}$ are independent observations with variance $\sigma^{2}$ then the variance of the mean $\bar{Z}$ is given by $\frac{\sigma^{2}}{n}$. 

__How is this relevant now?__



We will use this idea calculate $$ \hat{f}^{1}(x), \cdots, \hat{f}^{B}(x)$$ where each $\hat{f}^{i}$ represents a decision tree fitted to the bootstrapped data.

Then we will make a prediction by: 

$$ \hat{f}_{\text{avg}}(x) = \frac{1}{B}\sum_{b=1}^{B} \hat{f}^{b}(x)$$

Note that this is for regression and for the classification we can get majority vote.

[sklearn averages over probabilities not majority vote](https://scikit-learn.org/stable/modules/ensemble.html#forest)


## Random Forests

__Problem__ We still have some problem with this approach and random forests will address this problem. Can you see the issue?

- If we have a strong predictor then this will dominate in each tree.

Hint: Correlated trees

## Sklearn for RandomForests

In [1]:
import pandas as pd

In [2]:
## you can download the data from -- https://www.kaggle.com/ishaanv/ISLR-Auto#Heart.csv

## or http://faculty.marshall.usc.edu/gareth-james/ISL/data.html
heart = pd.read_csv('data/Heart.csv', index_col = 0)
heart.head()
print(heart.shape)

(303, 14)


In [11]:
heart.dropna(axis= 0, how= 'any', inplace = True)
y = heart.AHD
heart.drop(columns= 'AHD', inplace = True)

AttributeError: 'DataFrame' object has no attribute 'AHD'

In [20]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 1 to 302
Data columns (total 13 columns):
Age          297 non-null int64
Sex          297 non-null int64
ChestPain    297 non-null object
RestBP       297 non-null int64
Chol         297 non-null int64
Fbs          297 non-null int64
RestECG      297 non-null int64
MaxHR        297 non-null int64
ExAng        297 non-null int64
Oldpeak      297 non-null float64
Slope        297 non-null int64
Ca           297 non-null float64
Thal         297 non-null object
dtypes: float64(2), int64(9), object(2)
memory usage: 32.5+ KB


In [13]:
display(heart.Thal.value_counts())
display(heart.ChestPain.value_counts())

normal        164
reversable    115
fixed          18
Name: Thal, dtype: int64

asymptomatic    142
nonanginal       83
nontypical       49
typical          23
Name: ChestPain, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split 
X_train, X_test,y_train, y_test = train_test_split(heart, y, test_size= 0.20, stratify = y)

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
categorical_variables = X_train.select_dtypes(include=['object']).columns
numerical_variables = X_train.select_dtypes(include = ['int64', 'float64']).columns

Index(['Age', 'Sex', 'RestBP', 'Chol', 'Fbs', 'RestECG', 'MaxHR', 'ExAng',
       'Oldpeak', 'Slope', 'Ca'],
      dtype='object')

In [21]:
import numpy as np

In [22]:
ohe = OneHotEncoder(drop = 'first')
X_categ = ohe.fit_transform(X_train[categorical_variables]).toarray()
X_num = X_train[numerical_variables].values
Xtrain = np.concatenate((X_categ, X_num), axis = -1,)
Xtrain.shape

(237, 16)

In [27]:
## now we should transform the test data 
## to be able to use it for the prediction

X_test_categ = ohe.transform(X_test[categorical_variables]).toarray()
X_test_num = X_test[numerical_variables].values
Xtest = np.concatenate((X_test_categ, X_test_num), axis = -1,)
Xtest.shape

(60, 16)

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
clf = RandomForestClassifier(n_estimators= 100, 
                             criterion= 'gini', 
                             max_features= 'auto',
                             oob_score= True)

__Your Turn__

- Use 5 fold cross_validation to fit random forest classifier we created above.
- Don't forget to return training scores and trained estimators.

In [30]:
from sklearn.model_selection import cross_validate

In [51]:
# %load -r 1-6 supplement.py
validator = cross_validate(clf,
                                 Xtrain,
                                 y_train,
                                 return_train_score=True,
                                 return_estimator= True,
                                 cv=5)

In [52]:
cv

{'fit_time': array([0.09709382, 0.14613533, 0.15302682, 0.14864349, 0.10674238]),
 'score_time': array([0.00702929, 0.01396251, 0.00997019, 0.01187468, 0.00894308]),
 'estimator': (RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                         max_depth=None, max_features='auto', max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=100,
                         n_jobs=None, oob_score=True, random_state=None,
                         verbose=0, warm_start=False),
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                         max_depth=None, max_features='auto', max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
            

__Your Turn__

- What is the type of validator above?

- Check test vs train(validation) scores.

- Print "mean +/- std" for both train and test scores

- Also print oob_scores and compare them with cross_validation scores

In [53]:
# %load -r 9-17 supplement.py
mean_test = np.mean(validator['test_score'])
std_test = np.std(validator['test_score'])

mean_train = np.mean(validator['train_score'])
std_train = np.std(validator['train_score'])


print("Train score: %.3f +/- %.3f"%(mean_train, std_train))
print("Test score: %.3f +/- %.3f"%(mean_test, std_test))

Train score: 1.000 +/- 0.000
Test score: 0.831 +/- 0.063


In [54]:
clf.fit(Xtrain, y_train)
clf.oob_score_

0.8059071729957806

__Your Turn__

- Note that we have over-fitting problem. 

- Let's try to reduce over-fitting

In [92]:
# %load -r 20-27 supplement.py
clf = RandomForestClassifier(n_estimators= 10000,
                             criterion= 'gini',
                             max_features= 'auto', max_depth= 2, warm_start = True, 
                             oob_score= True)

clf.fit(Xtrain, y_train);
clf.score(Xtrain, y_train)
clf.oob_score_

0.8143459915611815

### Extra: Pipelines?

In [18]:
## There is an "easier" way to do this
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [29]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))])

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numerical_variables),
        ('cat', categorical_transformer, categorical_variables)],)

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators =100,
                                                            oob_score = True))])

In [58]:
pipe_validator = cross_validate(rf, 
                    X_train, 
                    y_train, 
                    return_train_score= True, 
                    return_estimator= True,
                    cv = 5)

In [31]:
## train scores
print(cv['train_score'])
## validation scores
print(cv['test_score'])

## let's pick one of the estimator for further investigation

est = cv['estimator'][0]

[1. 1. 1. 1. 1.]
[0.83333333 0.8125     0.70833333 0.78723404 0.86956522]


In [216]:
est['classifier'].oob_score_

0.6931216931216931

## Continue...

In [94]:
feature_importances = clf.feature_importances_

In [96]:
feature_importances

array([2.18436435e-02, 6.74276952e-03, 7.66920161e-03, 2.25398488e-01,
       1.45910462e-01, 2.16902591e-02, 5.65078638e-03, 1.47421665e-02,
       8.56973288e-03, 1.83589149e-04, 5.68561985e-03, 1.02895964e-01,
       1.32116711e-01, 1.08741969e-01, 5.92967992e-02, 1.32861838e-01])

In [98]:
# be careful with the order of columns
columns = ohe.get_feature_names().tolist() + numerical_variables.tolist()

In [99]:
importances = pd.DataFrame(data= feature_importances, index = columns, columns= ['feature_importances'])

importances.sort_values(by = 'feature_importances', ascending = False)

,feature_importances
x1_normal,0.225398
x1_reversable,0.145910
Ca,0.132862
ExAng,0.132117
Oldpeak,0.108742
MaxHR,0.102896
Slope,0.059297
x0_nonanginal,0.021844
Age,0.021690
RestBP,0.014742


### Extra Material 

- [Sklearn averages probabilities in RF implementation](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [On the variance](https://newonlinecourses.science.psu.edu/stat414/node/167/)

- [Do RF immune to overfitting?](https://en.wikipedia.org/wiki/Talk%3ARandom_forest)

- [Tricky stuff with respect to feature importance](http://rnowling.github.io/machine/learning/2015/08/10/random-forest-bias.html)

- [An interesting implementation of feature importance](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances_faces.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-faces-py)

- [Different Ensemble Methods in sklearn](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [ISLR - section 8.2](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf)

- [Another library for RF: H2o](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html)